<a href="https://colab.research.google.com/github/DmitryKutsev/DeepHW/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install youtokentome

     |████████████████████████████████| 1.7MB 2.8MB/s 


In [4]:
import math
import numpy as np
import pandas as pd

import random
import json
import torch
from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from matplotlib import pyplot as plt

import youtokentome as yttm

In [10]:
!unzip '/content/drive/My Drive/unsupervised.csv.zip'

Archive:  /content/drive/My Drive/unsupervised.csv.zip
  inflating: unsupervised.csv        


In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!ls

drive  sample_data  unsupervised.csv


In [14]:
questions = pd.read_csv('unsupervised.csv')
questions.question = questions.question.map(lambda x: x.lower())